In [53]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Sample product Data with genres and ratings

product_data = {
    'product_id' : [1, 2, 3, 4, 5],
    'title' : ['prod a', 'prod b', 'prod c', 'prod d', 'prod e'],
    'department' : ['elctronics', 'fashion', 'toys', 'stationary', 'soaps']
}

# Sample user ratings data
ratings_data = {
    'user_id': [1, 1, 2, 2, 3],
    'product_id': [1, 2, 3, 4, 5],
    'rating': [5, 4, 4, 3, 5]
}

# Convert product and ratings data to pandas DataFrame
product_df = pd.DataFrame(product_data)
ratings_df = pd.DataFrame(ratings_data)

# Create a user product rating metrix
user_prod_ratings = ratings_df.pivot(index = 'user_id',
                                      columns = 'product_id',
                                      values = 'rating').fillna(0)
print(user_prod_ratings)
# Calculate product similarities based on generes
generes_similarity = cosine_similarity(user_prod_ratings.values.T)
print(generes_similarity)

# Function to get product recommendations based on ratings
def get_prod_recommendations(product_id, num_recommendations=5):
    similar_products = list(enumerate(generes_similarity[product_id]))
    sorted_similar_products = sorted(similar_products, key=lambda x: x[1], reverse=True)
    recommended_products = []

    for product in sorted_similar_products[1:num_recommendations + 1]:
        product_id, similarity_score = product
        matching_products = product_df.loc[product_df['product_id'] == product_id, 'title'].values

        # Check if there are matching products before accessing the first element
        if len(matching_products) > 0:
            recommended_products.append(matching_products[0])

    return recommended_products

# Sample user's product ratings
user_id = 1

# Get unrated product for the user
unrated_products = [product_id for product_id in product_df['product_id'].values
                    if product_id not in user_prod_ratings.columns]

# Get product recommendations for the user
recommended_products = get_prod_recommendations(product_id=1, num_recommendations=3)

print('Recommended products for user {} are: {}'.format(user_id, recommended_products))


product_id    1    2    3    4    5
user_id                            
1           5.0  4.0  0.0  0.0  0.0
2           0.0  0.0  4.0  3.0  0.0
3           0.0  0.0  0.0  0.0  5.0
[[1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 1.]]
Recommended products for user 1 are: ['prod a', 'prod b', 'prod c']


### **Reading Dataset**

### **checking for the size**

In [54]:
data = pd.read_csv('/content/All Electronics.csv')

In [55]:
data.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999"
1,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/71AvQd3Vzq...,https://www.amazon.in/OnePlus-Nord-Lite-128GB-...,4.3,"113,956","₹18,999","₹19,999"
2,OnePlus Bullets Z2 Bluetooth Wireless in Ear E...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/51UhwaQXCp...,https://www.amazon.in/Oneplus-Bluetooth-Wirele...,4.2,"90,304","₹1,999","₹2,299"
3,"Samsung Galaxy M33 5G (Mystique Green, 6GB, 12...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81I3w4J6yj...,https://www.amazon.in/Samsung-Mystique-Storage...,4.1,"24,863","₹15,999","₹24,999"
4,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/71V--WZVUI...,https://www.amazon.in/OnePlus-Nord-Black-128GB...,4.3,"113,956","₹18,999","₹19,999"


In [56]:
data.shape

(9600, 9)

### **Checking for null values**

In [57]:
data.isnull().sum()

name                0
main_category       0
sub_category        0
image               0
link                0
ratings            95
no_of_ratings      95
discount_price    484
actual_price       70
dtype: int64

The Surprise library (Simple Python Recommendation System Engine) is a Python library designed for building and evaluating recommendation systems. It provides a simple and easy-to-use interface for implementing collaborative filtering algorithms and evaluating their performance


In [58]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

In [59]:
# Loading dataset
file_path = '/content/All Electronics.csv'
df = pd.read_csv(file_path)

# Add a placeholder user column
df['user'] = 'user_placeholder'

# Convert 'ratings' column to numeric and drop rows with NaN values
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
df = df.dropna(subset=['ratings'])

# Check for NaN values in 'ratings' column again
if df['ratings'].isnull().any():
    print("Warning: NaN values found in the 'ratings' column after dropping rows. Please inspect the data.")

# Define the rating scale (assuming ratings are on a scale from 1 to 5)
reader = Reader(rating_scale=(1, 5))

# Load the dataset into Surprise
data = Dataset.load_from_df(df[['user', 'name', 'ratings']], reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Build the recommendation model (SVD algorithm)(Singular Value Decomposition )
model = SVD()
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Evaluate the model (Root Mean Squared Error)
accuracy.rmse(predictions)

# Function to get recommendations for a given product
def get_recommendations(product_name):
    # Get the list of all products
    all_products = df['name'].unique()

    # Remove the product for which recommendations are needed
    other_products = [p for p in all_products if p != product_name]

    # Create a test set for the selected product and other products
    test_data = [('user_placeholder', other_product, 4.0) for other_product in other_products]

    # Make predictions on the test set
    test_predictions = model.test(test_data)

    # Sort predictions by estimated rating
    sorted_predictions = sorted(test_predictions, key=lambda x: x.est, reverse=True)

    # Get top N recommendations
    top_n = 5
    top_recommendations = [pred.iid for pred in sorted_predictions[:top_n]]

    return top_recommendations

# Example: Get recommendations for a product
product_to_recommend = 'Product1'
recommendations = get_recommendations(product_to_recommend)

print(f"Top Recommendations for {product_to_recommend}:")
for i, product_id in enumerate(recommendations, start=1):
    print(f"{i}. Product ID: {product_id}")


RMSE: 0.3724
Top Recommendations for Product1:
1. Product ID: Yogi MART Foldable Wooden Laptop Bed Tray Table, Multifunction Lap Tablet Desk with Cup Holder, Perfect for Eating Breakfa...
2. Product ID: JIYAMART Foldable Wooden Laptop Bed Tray Table, Multifunction Lap Tablet Desk with Cup Holder, Perfect for Eating Breakfas...
3. Product ID: Logitech Pebble Wireless Mouse with Bluetooth or 2.4 GHz Receiver, Silent, Slim Computer Mouse with Quiet Clicks, for Lapt...
4. Product ID: Amkette Pocket Mate FM Radio with Bluetooth Speaker - Type C Charging, Antenna, Multiple Playback,12 Hrs Playtime, and Num...
5. Product ID: LTZ Foldable Mobile Stand for Wall | Multi-Purpose Mobile Holder Charging Stand with 3 pin Plug | Phone Stand with Univers...


In [60]:
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,user
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999",user_placeholder
1,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/71AvQd3Vzq...,https://www.amazon.in/OnePlus-Nord-Lite-128GB-...,4.3,"113,956","₹18,999","₹19,999",user_placeholder
2,OnePlus Bullets Z2 Bluetooth Wireless in Ear E...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/51UhwaQXCp...,https://www.amazon.in/Oneplus-Bluetooth-Wirele...,4.2,"90,304","₹1,999","₹2,299",user_placeholder
3,"Samsung Galaxy M33 5G (Mystique Green, 6GB, 12...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81I3w4J6yj...,https://www.amazon.in/Samsung-Mystique-Storage...,4.1,"24,863","₹15,999","₹24,999",user_placeholder
4,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/71V--WZVUI...,https://www.amazon.in/OnePlus-Nord-Black-128GB...,4.3,"113,956","₹18,999","₹19,999",user_placeholder
